In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [2]:
STORM_NUMBER = 9
TITLE = "29-30 December 2020"
SUBTITLE = "8 AM 30 December 2020"
sts = utc(2020, 12, 29, 20)
ets = utc(2020, 12, 30, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 20, 'z': 'no', 'f': 'linear'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [23]:
cull = ['IA-DL-33', 144, 475, 'WMTI4', 203, 'RKWI4', 'ALXI4', 635, 695, 648, 1059, 1018, 1009, 977, '3OI']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
    ['AMOI4', 8],
    ['IA-PA-5', 1]
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=1)
res = mp.postprocess(filename='201230.png')
mp.close()

     state  wfo     val      lon      lat                             geo  \
1079    WI  ARX  0.8000 -91.2600  43.8700   POINT (698970.665 -88064.625)   
1686    MO  LSX  0.8000 -91.8872  39.9714  POINT (690634.382 -525103.964)   
1711    MO  LSX  0.5000 -91.5200  40.1400  POINT (719929.037 -503305.640)   
1716    IL  LSX  0.5000 -91.6000  39.9058  POINT (715685.978 -529926.375)   
1732    IL  ILX  0.4000 -90.4639  40.5914  POINT (803584.004 -443610.148)   
1765    IA  DVN  0.2000 -91.3667  40.4000  POINT (730000.687 -473178.864)   
1780    MN  MPX  0.2000 -93.7364  43.7462  POINT (502555.941 -120076.303)   
1872    IL  ILX  0.0001 -90.2289  39.7136  POINT (834276.358 -538352.857)   

      used_for_analysis  nwsli  plotme source  
1079               True   1079    True    LSR  
1686               True  SFFM7    True   COOP  
1711               True  CANM7    True   COOP  
1716               True  QLDI2    True   COOP  
1732               True  PRCI2    True   COOP  
1765              